In [18]:
import numpy as np
import pandas as pd

In [19]:
from IPython.display import Image

In [33]:
np.random.seed(666)
a = np.random.randint(1,100,100)

In [35]:
b = a.reshape(20,-1)

In [36]:
b

array([[ 3, 46, 31, 63, 71],
       [74, 31, 37, 62, 92],
       [95, 52, 61, 96, 29],
       [15, 98, 64, 17, 47],
       [40, 70, 83, 77, 80],
       [14, 70, 21, 12, 25],
       [21, 37, 49, 37, 95],
       [47, 48, 17, 78, 37],
       [27,  6, 43, 46, 34],
       [65, 95, 65, 28, 15],
       [39, 65, 93, 84,  3],
       [92, 35, 87, 28, 80],
       [39, 28, 81, 58, 84],
       [39, 10, 52, 78, 46],
       [ 3, 89, 64, 24, 12],
       [72, 55, 85, 48, 20],
       [41, 48, 22, 94, 84],
       [ 6, 26, 90, 70,  5],
       [13, 61, 59, 19, 70],
       [21, 53, 26, 64, 92]])

In [37]:
from sklearn.preprocessing import StandardScaler

In [38]:
scaler = StandardScaler()

In [43]:
c = scaler.fit_transform(b)

/Users/jing/.virtualenvs/pydata/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [44]:
c

array([[-1.27443467, -0.20697466, -1.04360182,  0.34113546,  0.63400539],
       [ 1.28887586, -0.80981348, -0.79804845,  0.30258908,  1.30137949],
       [ 2.04703813,  0.03416087,  0.18416503,  1.61316599, -0.7007428 ],
       [-0.84119909,  1.88286658,  0.30694171, -1.43199801, -0.12870786],
       [ 0.06137504,  0.75756745,  1.08452738,  0.88078478,  0.92002286],
       [-0.87730205,  0.75756745, -1.45285744, -1.6247299 , -0.82786168],
       [-0.6245813 , -0.56867795, -0.30694171, -0.66107041,  1.39671865],
       [ 0.3140958 , -0.12659615, -1.61655969,  0.91933116, -0.44650505],
       [-0.40796351, -1.81454485, -0.55249508, -0.31415299, -0.54184421],
       [ 0.96394917,  1.76229882,  0.34786727, -1.00798783, -1.14565887],
       [ 0.02527208,  0.55662118,  1.493783  ,  1.15060943, -1.5270155 ],
       [ 1.93872923, -0.64905646,  1.24822963, -1.00798783,  0.92002286],
       [ 0.02527208, -0.93038125,  1.00267626,  0.14840356,  1.04714174],
       [ 0.02527208, -1.65378783, -0.1

In [46]:
scaler.inverse_transform(c)

array([[ 3., 46., 31., 63., 71.],
       [74., 31., 37., 62., 92.],
       [95., 52., 61., 96., 29.],
       [15., 98., 64., 17., 47.],
       [40., 70., 83., 77., 80.],
       [14., 70., 21., 12., 25.],
       [21., 37., 49., 37., 95.],
       [47., 48., 17., 78., 37.],
       [27.,  6., 43., 46., 34.],
       [65., 95., 65., 28., 15.],
       [39., 65., 93., 84.,  3.],
       [92., 35., 87., 28., 80.],
       [39., 28., 81., 58., 84.],
       [39., 10., 52., 78., 46.],
       [ 3., 89., 64., 24., 12.],
       [72., 55., 85., 48., 20.],
       [41., 48., 22., 94., 84.],
       [ 6., 26., 90., 70.,  5.],
       [13., 61., 59., 19., 70.],
       [21., 53., 26., 64., 92.]])

In [52]:
scaler.fit_transform(b[:5])

/Users/jing/.virtualenvs/pydata/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.22101504, -0.58280704, -1.27516865,  0.        ,  0.31557101],
       [ 0.8236092 , -1.23520298, -0.95901113, -0.03833698,  1.23598646],
       [ 1.42835722, -0.32184866,  0.30561893,  1.26512019, -1.52525988],
       [-0.87544475,  1.67883221,  0.46369769, -1.76350087, -0.73633236],
       [-0.15550663,  0.46102646,  1.46486316,  0.53671766,  0.71003477]])

In [53]:
b[:5]

array([[ 3, 46, 31, 63, 71],
       [74, 31, 37, 62, 92],
       [95, 52, 61, 96, 29],
       [15, 98, 64, 17, 47],
       [40, 70, 83, 77, 80]])

In [55]:
d = b[:5]

In [68]:
d.mean(axis=0)

array([45.4, 59.4, 55.2, 63. , 63.8])

In [69]:
d[:,0]

array([ 3, 74, 95, 15, 40])

In [70]:
d.std(axis=0)

array([34.72520698, 22.99217258, 18.97788186, 26.08447814, 22.81578401])

In [72]:
s = (d-d.mean(axis=0))/d.std(axis=0)

In [73]:
s

array([[-1.22101504, -0.58280704, -1.27516865,  0.        ,  0.31557101],
       [ 0.8236092 , -1.23520298, -0.95901113, -0.03833698,  1.23598646],
       [ 1.42835722, -0.32184866,  0.30561893,  1.26512019, -1.52525988],
       [-0.87544475,  1.67883221,  0.46369769, -1.76350087, -0.73633236],
       [-0.15550663,  0.46102646,  1.46486316,  0.53671766,  0.71003477]])

In [74]:
s*(d.std(axis=0))+(d.mean(axis=0))

array([[ 3., 46., 31., 63., 71.],
       [74., 31., 37., 62., 92.],
       [95., 52., 61., 96., 29.],
       [15., 98., 64., 17., 47.],
       [40., 70., 83., 77., 80.]])